# AIカメラを使って親しいひとかどうか判断

相手が親しい人だと犬は喜びますが、知らない人だと吠えたり逃げますよね？　LEGO DOGにも同じ動きをさせましょう。AIカメラには顔識別機能がありますので、親しい人かどうか、顔を見て判断することができます。顔認識は最初の授業の自動とびらで使いました。AIカメラの顔の学習方法は＃＃＃を見てください。

知ってる人かどうか顔を見て判断するプログラムは以下です。もし学習済みの顔の場合、ピッと音を鳴らしてスマイルマークを表示します。知らない人の場合は×を表示します。顔が見えない場合は？を表示します。

In [ ]:
import sys
import time
from hub import port

from spike import PrimeHub
prime_hub = PrimeHub()        # HUBの初期化

sys.path.append('/projects/mylib000')
from huskylens_lib import Algo
from huskylens_lib import HuskyLens

port.F.mode(port.MODE_FULL_DUPLEX)
time.sleep(1)
port.F.baud(9600)

husky = HuskyLens(port.F)        # AIカメラ初期化
husky.send_CMD_REQ_ALGO(Algo.FACE_RECOGNITION)# 顔認識モードに設定

prime_hub.light_matrix.write('?')
while True:
    blocks = husky.read_blocks()
    if len(blocks) == 0:
         prime_hub.light_matrix.write('?')
    else:
        print(blocks)
        if blocks[0][1] == 1:
            print('found')
            prime_hub.speaker.beep(note=85, seconds=0.2)
            prime_hub.light_matrix.show_image('HAPPY')
        else:
            print('unnown person')
            prime_hub.speaker.beep(note=60, seconds=0.2)
            prime_hub.light_matrix.write('X')

カメラに近づきすぎると顔全体が写らなくなり、正しく認識されません。カメラから 20 cmぐらい離れて、カメラの前に顔を置いてください。
なお、学習済みの顔と判断した場合、カメラの上に付いているLEDが緑色に点灯します。顔が映っているが、知らない人の顔の場合、LEDは白色に点灯します。

# LEGO DOGのプログラムに組み込み

これまでのLEGO DOGの動きは以下でした

1.   人を探す
2.   人を見つけたら近づく(50cm進む)

このプログラムに、近づいた後、AIカメラを使って親しい人かどうかを判断するプログラムを追加しましょう。動きとしては以下となります。

1.   人を探す
2.   人を見つけたら近づく(50cm進む)
3. 親しい人かどうか判断する
4. 親しい人だったら、うれしい動作をする
5. 知らない人だったら、怒ったり、逃げる動作をする


まずは、１～３までを作りましょう。先ほどの近づくプログラムに、さらに知っている人かどうかを判断するプログラムを加えます。全体のプログラムは以下となります

In [ ]:
#
#LEGO DOG
#機能
#    (1) 人を探す
#    (2) 人を見つけたら前進する
#    (3) 知っている人か判断する
#

import sys
import time
from hub import port
from spike import MotorPair#モータを使う


sys.path.append('/projects/mylib000')
from huskylens_lib import Algo
from huskylens_lib import HuskyLens

from spike import PrimeHub

prime_hub = PrimeHub()        # HUBの初期化
motor_pair = MotorPair('A','B')# モータペアの初期化
motor_pair.set_motor_rotation(17.6, 'cm')    #タイヤのサイズを指定します

port.F.mode(port.MODE_FULL_DUPLEX)
time.sleep(1)
port.F.baud(9600)

husky = HuskyLens(port.F)

husky.send_CMD_REQ_ALGO(Algo.OBJECT_RECOGNITION)
time.sleep(1)

while True:
    blocks = husky.read_blocks()
    if len(blocks) == 0:
        prime_hub.light_matrix.off()
    else:
        print(blocks)
        if blocks[0][1] == 1:
            print('found')
            prime_hub.speaker.beep(note=80, seconds=0.2)
            prime_hub.light_matrix.show_image('HAPPY')
            motor_pair.move(30, unit='cm', steering=0,speed=40)# 30cmまっすぐ進むようmove関数を実行

            # 親しい人かどうかを判断する　（追加するプログラム）
            husky.send_CMD_REQ_ALGO(Algo.FACE_RECOGNITION)# 顔認識モードに設定
            time.sleep(1)    # 切り替えが終わるまで少し待つ
            prime_hub.light_matrix.write('h')
            while True:
                blocks = husky.read_blocks()
                if len(blocks) == 0:
                    prime_hub.light_matrix.write('?')
                else:
                    print(blocks)
                    if blocks[0][1] == 1:
                        print('found')
                        prime_hub.speaker.beep(note=85, seconds=0.2)
                        prime_hub.light_matrix.show_image('HAPPY')
                    else:
                        print('unnown person')
                        prime_hub.speaker.beep(note=60, seconds=0.2)
                        prime_hub.light_matrix.write('X')

上記プログラムは、人をみつけたら近づくコードに、顔を判断するコードをくっつけました。プログラム全体が複雑になっているので、もう少し分かりやすくコードを作り変えましょう。出来上がったコードを作り変えることをリファクタリングと呼びます。

In [ ]:
#
#LEGO DOG
#機能
#    (1) 人を探す
#    (2) 人を見つけたら前進する
#    (3) 知っている人か判断する
#

import sys
import time
from hub import port
from spike import MotorPair#モータを使う


sys.path.append('/projects/mylib000')
from huskylens_lib import Algo
from huskylens_lib import HuskyLens

from spike import PrimeHub

prime_hub = PrimeHub()        # HUBの初期化
motor_pair = MotorPair('A','B')# モータペアの初期化
motor_pair.set_motor_rotation(17.6, 'cm')    #タイヤのサイズを指定します

port.F.mode(port.MODE_FULL_DUPLEX)
time.sleep(1)
port.F.baud(9600)

husky = HuskyLens(port.F)

#
# 人を探す。見つけたら前進する
#
husky.send_CMD_REQ_ALGO(Algo.OBJECT_RECOGNITION)
prime_hub.light_matrix.write('R')             # モノ認識であることを分かるようにする（デバッグ用）
time.sleep(1)   # 切り替えが終わるまで少し待つ
while True:
    blocks = husky.read_blocks()
    if len(blocks) == 0:
        prime_hub.light_matrix.off()
    else:
        print(blocks)
        if blocks[0][1] == 1:
            print('found')
            prime_hub.speaker.beep(note=80, seconds=0.2)
            prime_hub.light_matrix.show_image('HAPPY')
            motor_pair.move(30, unit='cm', steering=0,speed=40)# 30cmまっすぐ進むようmove関数を実行
            break

#
# 親しい人かどうかを判断する
#
husky.send_CMD_REQ_ALGO(Algo.FACE_RECOGNITION)# 顔認識モードに設定
prime_hub.light_matrix.write('F')             # 顔を探すことを分かるようにする（デバッグ用）
time.sleep(1)    # 切り替えが終わるまで少し待つ
while True:
    blocks = husky.read_blocks()
    if len(blocks) == 0:
        prime_hub.light_matrix.write('?')
    else:
        print(blocks)
        if blocks[0][1] == 1:
            print('found')
            prime_hub.speaker.beep(note=85, seconds=0.2)
            prime_hub.light_matrix.show_image('HAPPY')
        else:
            print('unnown person')
            prime_hub.speaker.beep(note=60, seconds=0.2)
            prime_hub.light_matrix.write('X')



上記プログラムでは、人を見つけて前進したらループを抜けて、次の処理（知っている人かどうか）に進むように改良しています

# 親しい人だったらうれしい気持ち、知らない人だったら怒る気持ちを表現しよう

飼っている犬だったら、嬉しい、怒る、の気持ちを動作で示します。LEGOを使って音や動きで感情を表現してみましょう。音を鳴らして表現する方法として、犬の鳴き声があります。４種類の鳴き声が入っています。
- 'Dog Bark 1',
- 'Dog Bark 2',
- 'Dog Bark 3',
- 'Dog Whining 1'
- 'Dog Whining 2'

Pythonのfor文を使って順番にならしてみましょう。

In [ ]:
from spike import App
app = App()

for sound in ('Dog Bark 1', 'Dog Bark 2', 'Dog Bark 3', 'Dog Whining 1', 'Dog Whining 2'):
   print(sound)
   app.play_sound(sound)

どれも似たり寄ったりであまり親しい感じ、怒っている感じがよく分かりませんが、、、

知らない人に出会って怒っている感じの鳴き方例

In [ ]:
from spike import App
app = App()
import time

app.play_sound('Dog Bark 1')
time.sleep(0.5)
app.play_sound('Dog Bark 1')
app.play_sound('Dog Bark 1')
time.sleep(0.2)
app.play_sound('Dog Bark 1')

親しい人に出会った時の鳴き方例

In [ ]:
from spike import App
app = App()
import time

app.play_sound('Dog Bark 3')
time.sleep(0.2)
app.play_sound('Dog Whining 1')

他にもいろいろ音は入っているので試してください。ひとまず、知ってる人ー＞うれしい、知らない人ー＞怒る　の組み合わせで、鳴く動作を加えます。鳴き声を組み込んだソースコードは以下となります。

In [ ]:
#
#LEGO DOG
#機能
#    (1) 人を探す
#    (2) 人を見つけたら前進する
#    (3) 知っている人か判断する
#    (4) 知っている人ならうれしい鳴き声、知らない人なら怒った鳴き声を出す
#

import sys
import time
from hub import port
from spike import MotorPair    #モータを使う
from spike import App          # Applicationを使う




sys.path.append('/projects/mylib000')
from huskylens_lib import Algo
from huskylens_lib import HuskyLens

from spike import PrimeHub

prime_hub = PrimeHub()        # HUBの初期化
motor_pair = MotorPair('A','B')# モータペアの初期化
motor_pair.set_motor_rotation(17.6, 'cm')    #タイヤのサイズを指定します

port.F.mode(port.MODE_FULL_DUPLEX)
time.sleep(1)
port.F.baud(9600)

husky = HuskyLens(port.F)

app = App()  # applicationの初期化


#
# 人を探す。見つけたら前進する
#
husky.send_CMD_REQ_ALGO(Algo.OBJECT_RECOGNITION)
prime_hub.light_matrix.write('R')            # モノ認識であることを分かるようにする（デバッグ用）
time.sleep(1)# 切り替えが終わるまで少し待つ
while True:
    blocks = husky.read_blocks()
    if len(blocks) == 0:
        prime_hub.light_matrix.off()
    else:
        print(blocks)
        if blocks[0][1] == 1:
            print('found')
            prime_hub.speaker.beep(note=80, seconds=0.2)
            prime_hub.light_matrix.show_image('HAPPY')
            motor_pair.move(30, unit='cm', steering=0,speed=40)# 30cmまっすぐ進むようmove関数を実行
            break

#
# 親しい人かどうかを判断する
#
husky.send_CMD_REQ_ALGO(Algo.FACE_RECOGNITION)# 顔認識モードに設定
prime_hub.light_matrix.write('F')            # 顔を探すことを分かるようにする（デバッグ用）
time.sleep(1)    # 切り替えが終わるまで少し待つ
while True:
    blocks = husky.read_blocks()
    if len(blocks) == 0:
        prime_hub.light_matrix.write('?')
    else:
        print(blocks)
        if blocks[0][1] == 1:
            print('found')
            prime_hub.speaker.beep(note=85, seconds=0.2)
            prime_hub.light_matrix.show_image('HAPPY')
            app.play_sound('Dog Bark 3')
            time.sleep(0.2)
            app.play_sound('Dog Whining 1')

        else:
            print('unnown person')
            prime_hub.speaker.beep(note=60, seconds=0.2)
            prime_hub.light_matrix.write('X')

            app.play_sound('Dog Bark 1')
            time.sleep(0.5)
            app.play_sound('Dog Bark 1')
            app.play_sound('Dog Bark 1')
            time.sleep(0.2)
            app.play_sound('Dog Bark 1')



動かしてみてください。正しく動くでしょうか？？　知っている人が前に来ると構ってほしい鳴き声になって、知らない人だと怒る鳴き声になりますが正しく動作したでしょうか。

ソースコードがまた複雑になってきたので、ソースコードを見やすくリファクタリングします。鳴き声を関数として分けてみます。
file:  LEGO_DOG_v1.llsp

In [ ]:
#
#LEGO DOG (全機能入り V1.00) (2025/6/13)
#機能
#    (1) 人を探す
#    (2) 人を見つけたら前進する
#    (3) 知っている人か判断する
#    (4) 知っている人ならうれしい鳴き声、知らない人なら怒った鳴き声を出す
#        w/ refactoring
#

import sys
import time
from hub import port
from spike import MotorPair    #モータを使う
from spike import App          # Applicationを使う


sys.path.append('/projects/mylib000')
from huskylens_lib import Algo
from huskylens_lib import HuskyLens

from spike import PrimeHub

prime_hub = PrimeHub()        # HUBの初期化
motor_pair = MotorPair('A','B')# モータペアの初期化
motor_pair.set_motor_rotation(17.6, 'cm')    #タイヤのサイズを指定します

port.F.mode(port.MODE_FULL_DUPLEX)
time.sleep(1)
port.F.baud(9600)

husky = HuskyLens(port.F)

app = App()  # applicationの初期化

# 犬の鳴き声（怒る）
def dog_bark_angry():
    app.play_sound('Dog Bark 1')
    time.sleep(0.5)
    app.play_sound('Dog Bark 1')
    app.play_sound('Dog Bark 1')
    time.sleep(0.2)
    app.play_sound('Dog Bark 1')

# 犬の鳴き声（親しい）
def dog_bark_happy():
    app.play_sound('Dog Bark 3')
    time.sleep(0.2)
    app.play_sound('Dog Whining 1')

#
# 人を探す。見つけたら前進する
#
husky.send_CMD_REQ_ALGO(Algo.OBJECT_RECOGNITION)
prime_hub.light_matrix.write('R')            # モノ認識であることを分かるようにする（デバッグ用）
time.sleep(1)# 切り替えが終わるまで少し待つ
while True:
    blocks = husky.read_blocks()
    if len(blocks) == 0:
        prime_hub.light_matrix.off()
    else:
        print(blocks)
        if blocks[0][1] == 1:
            print('found')
            prime_hub.speaker.beep(note=80, seconds=0.2)
            prime_hub.light_matrix.show_image('HAPPY')
            motor_pair.move(30, unit='cm', steering=0,speed=40)# 30cmまっすぐ進むようmove関数を実行
            break

#
# 親しい人かどうかを判断する
#
husky.send_CMD_REQ_ALGO(Algo.FACE_RECOGNITION)# 顔認識モードに設定
prime_hub.light_matrix.write('F')            # 顔を探すことを分かるようにする（デバッグ用）
time.sleep(1)    # 切り替えが終わるまで少し待つ
while True:
    blocks = husky.read_blocks()
    if len(blocks) == 0:
        prime_hub.light_matrix.write('?')
    else:
        print(blocks)
        if blocks[0][1] == 1:
            print('found')
            prime_hub.speaker.beep(note=85, seconds=0.2)
            prime_hub.light_matrix.show_image('HAPPY')
            dog_bark_happy()

        else:
            print('unnown person')
            prime_hub.speaker.beep(note=60, seconds=0.2)
            prime_hub.light_matrix.write('X')
            dog_bark_angry()




LEGO DOGのプログラムは以上で終わりです。もし授業の時間があったら、嬉しい動作として、サーボを使って前後に動く動作や、知らない人に出会ったら後ろに進む動作等を足してみてください。